# Manacher’s algorithm

長さ $N$ の文字列 $S$ の $i$ 番目の文字を中心とする最長回文の半径を求められる。
> https://snuke.hatenablog.com/entry/2014/12/02/235837

## 計算量
$O(N)$

## 実装
- 偶数長含めた回文の長さを求めるため、ダミー文字を挿入
- 各 $i$ ごとに回文半径 $R[i]$ を調べる
- 既知の対称性を利用し、探索を飛ばす

In [1]:
def Manacher(S: str):
    # ダミー文字を挿入
    dummy = "0"
    C = []
    for a in S:
        C.append(0)
        C.append(a)   
    C.append(0)

    L = len(C)
    R = [0] * L
    
    # 探索位置
    i = 0
    # 半径
    j = 0
    while i < L:
        # すでに回文であることがわかっている部分を飛ばす
        # 回文を探索
        while j <= i < L-j and C[i-j] == C[i+j]:
            j += 1
        R[i] = j
        k = 1
        # 対称性から、求めた回文半径の範囲内に存在する回文の答えを埋める
        while k <= i < L-k and k + R[i-k] < j:
            R[i+k] = R[i-k]
            k += 1
        # 探索開始位置をスライドする
        i += k
        # 開始位置を i にスライドした場合に、
        # すでにわかっている回文半径
        j -= k
    
    R = list(map(lambda x: x-1, R))
    # i=0,1,2,...,|S|-1
    # R[2*i+1] = L: S[i]  を中心とする奇数長の最大回文 
    # R[2*i] = L: S[i:i+2]を中心とする偶数長の最大回文
    # ダミー文字を挟むが、各 R[i] は実際の回文の文字列長と一致する
    return R

## [使用例 1](https://atcoder.jp/contests/ukuku09/tasks/ukuku09_d)
長さ $N$ の文字列 $S$ について、$S[l,r]$ の最長回文の長さを $Q$ 回答える。

In [2]:
# Q個のクエリを高速に処理するためのデータ構造
class SparseTable:
    def __init__(self, A):
        self.A = A
        N = len(A)
        # インデックス k として必要な最大値
        # 例: N = 8 の場合、2^2 まで計算できればよい
        max_k = (N-1).bit_length() - 1
        
        # DP: i=0,1,2,...,N-1 に対して、𝑘=0,1,...,max_k の値を順番に評価していく
        INF = 10**10
        # 𝑡𝑎𝑏𝑙𝑒[𝑖][𝑘] を初期化
        table = [[INF]*(max_k+1) for _ in range(N)]
        # table[i][k=0] = i で自明
        for i in range(N):
            table[i][0]=i
        
        # table[i][k]を求める際、table[i][k−1] の計算結果を利用する
        # さらに、table[i][k]を求める際、table[i+2^(k-1)][k−1] の計算結果を利用する 
        for k in range(1, max_k + 1):
            for i in range(N):
                if i + (1 << k) > N:
                    break
                # print(k,i,i+(1<<(k-1)))
                # k-1 で求めた結果を利用
                first = table[i][k-1]
                second = table[i+(1<<(k-1))][k-1]

                # table[i][k] の値を区間内の最小値を持つインデックスとする
                if A[first] <= A[second]:
                    table[i][k] = first
                else:
                    table[i][k] = second

        self.table = table

    # 区間[l,r] (0 <= l < r <= N-1) の最小値を持つインデックスを返す
    def query(self, l, r):
        # 同値の場合
        if l==r:
            return l
        # l > r の場合入れ替え
        if l > r:
            l, r = r, l
        # 区間の長さ
        d = r - l + 1
        k = (d-1).bit_length() - 1
        
        # A[l:r+2^k-1], A[r−2^k+1:r] の2つの区間の最小値を持つインデックスを返す
        min_idx_l = self.table[l][k]
        min_idx_r = self.table[r-(1<<k)+1][k]
        if self.A[min_idx_l] < self.A[min_idx_r]:
            return min_idx_l
        else:
            return min_idx_r

In [3]:
# 回答
S='acbcaaaddda'
Queries=[[5, 9], [9, 11], [7, 10], [6, 6], [8, 11]]

R=Manacher(S)

def solve_TLE():
    # Rを左から一つずつ調べるとO(NQ)だから問題によってはTLEする
    for l,r in Queries:
        ans=1
        l-=1
        r-=1
        min_l=l*2+1
        max_r=r*2+1
        for i in range(l,r+1,1):
            # 奇数長のみ
            idx=i*2+1
            # print((R[idx],idx-(min_l-1),max_r-(idx-1)))
            ans=max(min(R[idx],idx-(min_l-1),max_r-(idx-1)),ans)
        print(ans)
        
def solve():
    # 奇数長のみ、SparseTable を利用するため -1 を掛ける
    R_odd = [-1*R[2*i+1] for i in range(len(S))]
    st = SparseTable(R_odd)
        
    for l,r in Queries:
        l-=1
        r-=1
        if r-l < 2:
            print(1)
            continue
        # 答えとなる回文長を初期化
        ok = 1
        # 答えとして存在し得る最大回文長
        ng = 2*((r-l)//2)+1
        # print(r-l+1,(ok,ng))
        # 回文長 k を [ok:ng] 間で変化させ、答えで二分探索
        while ok < ng:
            # 長さ k の回文を作れるか
            k = (ok+ng)//2
            k = 2*(k//2)-1
            # 二分探索の左端
            # a = l+k-1
            a = l + k//2
            # 二分探索の右端
            # b = r-k+1
            b = r - k//2
            # 範囲内の最大回文長を取得
            _max = -1*R_odd[st.query(a,b)]
            # print((_max,k),(a,b))
            if _max > k:
                ok = k + 2
            if _max < k:
                ng = k
            if _max == k:
                ok = k
                break
            # print((ok,ng))
        
        # 最後に ok = k + 2 して抜けた時に範囲外になっていないか確かめる
        a = l + ok//2
        b = r - ok//2
        _max = -1*R_odd[st.query(a,b)]
        if _max>=ok:
            print(ok)
        else:
            print(ok-2)

In [4]:
S

'acbcaaaddda'

In [5]:
solve()

3
1
3
1
3


In [6]:
solve_TLE()

3
1
3
1
3


In [7]:
R_odd = [-1*R[2*i+1] for i in range(len(S))]
R_odd

[-1, -1, -5, -1, -1, -3, -1, -1, -5, -1, -1]